# Exploration des cahiers de doléance

In [4]:
import csv
import spacy

In [6]:
nlp = spacy.load('fr_core_news_sm')

In [2]:
with open('./resources/cahiers.csv') as f:
    data = list(csv.DictReader(f))

In [15]:
for item in data:
    doc = nlp(item['texte_cahier_txt'])
    item['sentences'] = [str(sent) for sent in doc.sents]

In [18]:
for item in data:
    for sent in item['sentences']:
        if 'sel' in sent:
            print(sent)
            print()

qu’enfin il plaise à sa majesté de permettre à ses procureurs généraux, comme à ses procureurs et substituts, il soit fait, à leur réquisition, une exacte recherche des différentes dilapidations et vols des deniers de l’état, et ceux qui seront atins et convaincus de cette indigne manœuvre punis selon la rigueur des loix, leurs biens vendus, et les deniers en provenant employés à aléger les dettes de l’état
Présidant l’assemblée : Feniou [12]
Greffier commis :

— Que dans le cas où le régime actuel de la gabelle seroit changé, comme il y a lieu de le présumer, les salines de Saintonge ne pouvant se soutenir qu’autant que les sels qui en proviennent auront un débouché assuré, le bon plaisir de Sa Majesté soit que les pays actuellement aprovisionnés des sels de nos salines, dont la qualité supérieure est générallement reconnue, seront tenus d’y prendre le sel nécessaire à leur consommation et de constater le chargement de leur navires dans nos ports par un certiftîcat du bureau de l’amir